Dataset is from a bar in Downtown. The data contains information on sales in high-end bar-cum-restaurant.  

In [2]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import openpyxl

In [4]:
mainSheet = pd.read_excel("Ann Driks Dashboard.xlsx",
                          sheet_name="Database",header=0)

In [5]:
mainSheet.head(2)

,Id,Date,Customers,Gender,Sales Reps,Drinks/Products,Categories,Cost,Sals Price,Qty,Cogs,Revenue,Profit
0,1,2019-01-01,Bernard Weatherly,M,timika poe,Heineken Bottle,Alcoholic Wine,38250,42500,3,114750,127500,12750
1,2,2019-01-02,Cordia Knopp,F,zona otis,Origin Bottle,Alcoholic Wine,2700,3000,5,13500,15000,1500
